In [84]:
import utils
from utils import collection

#### 1) Quel est le film le plus long ?

In [85]:
longest_movie_title = utils.longest_movie(collection)
print(longest_movie_title)

Le film le plus long est : ' Gone with the Wind ' avec une durée de 238 minutes.


#### 2) Quels sont les 5 films les mieux notés ?

In [86]:
top_film = utils.top_film_note(collection)
print(top_film)

Les cinq meilleurs films sont : ['The Shawshank Redemption', 'The Godfather', 'The Godfather Part II', '12 Angry Men', "Schindler's List"]


#### 3) Dans combien de films a joué Morgan Freeman ? Tom Cruise ?

In [87]:
count_movies_by_actor = utils.count_movies('Morgan Freeman')
print(count_movies_by_actor)

L'acteur ' Morgan Freeman ' a joué dans 4 films


In [88]:
count_movies_by_actor = utils.count_movies('Tom Cruise')
print(count_movies_by_actor)

L'acteur ' Tom Cruise ' a joué dans 0 films


#### 4) Quels sont les 3 meilleurs films d’horreur ? Dramatique ? Comique ?

In [89]:
top_movie_by_genre = utils.top_movies_by_genre('Drama', 5)
print(top_movie_by_genre)

Les 5 meilleurs films de genre ' Drama ' sont:
1 - The Shawshank Redemption
2 - The Godfather
3 - The Lord of the Rings: The Return of the King
4 - The Godfather Part II
5 - 12 Angry Men



In [90]:
top_movie_by_genre = utils.top_movies_by_genre('Comedy', 5)
print(top_movie_by_genre)

Les 5 meilleurs films de genre ' Comedy ' sont:
1 - Life Is Beautiful
2 - The Intouchables
3 - Modern Times
4 - Back to the Future
5 - City Lights



#### 5) Parmi les 100 films les mieux notés, quel pourcentage sont américains ? Français ?

In [91]:
percentage_by = utils.percentage_by_country('United States', 100)
print(percentage_by)

Parmi les 100 films les mieux notés, 62.00% sont de United States.


In [92]:
percentage_by_cntry = utils.percentage_by_country('French', 100)
print(percentage_by_cntry)

Parmi les 100 films les mieux notés, 0.00% sont de French.


#### 6) Quel est la durée moyenne d’un film en fonction du genre ?

In [93]:
average_time_by_genre = utils.average_time_by_genre(collection)
print(average_time_by_genre)

La durée moyenne d'un film de genre Action est de 133.98 minutes.
La durée moyenne d'un film de genre Adventure est de 130.12 minutes.
La durée moyenne d'un film de genre Animation est de 100.96 minutes.
La durée moyenne d'un film de genre Biography est de 145.50 minutes.
La durée moyenne d'un film de genre Comedy est de 105.60 minutes.
La durée moyenne d'un film de genre Crime est de 132.50 minutes.
La durée moyenne d'un film de genre Drama est de 134.50 minutes.
La durée moyenne d'un film de genre Family est de 114.54 minutes.
La durée moyenne d'un film de genre Fantasy est de 123.57 minutes.
La durée moyenne d'un film de genre Film-Noir est de 110.00 minutes.
La durée moyenne d'un film de genre History est de 152.11 minutes.
La durée moyenne d'un film de genre Horror est de 120.60 minutes.
La durée moyenne d'un film de genre Music est de 128.00 minutes.
La durée moyenne d'un film de genre Musical est de 103.00 minutes.
La durée moyenne d'un film de genre Mystery est de 124.61 minute

### * Questions Bonus : 

#### 7) En fonction du genre, afficher la liste des films les plus longs.

In [94]:
longest_movies_by_genre = utils.longest_movies_by_genre('Drama', 5)
print(longest_movies_by_genre)

Les 5 films les plus longs de genre 'Drama' sont : 
1. Gone with the Wind
2. Once Upon a Time in America
3. Lawrence of Arabia
4. Ben-Hur
5. Seven Samurai



#### 8) En fonction du genre, quel est le coût de tournage d’une minute de film ?

In [95]:
cost_per_minute_by_genre = utils.cost_per_minute_by_genre('Comedy')
print(cost_per_minute_by_genre)

26772779.721518986


#### 9) Quels sont les séries les mieux notés ?